# Boolean Operators — Practice Notebook

Hands-on exercises focusing on:
- Truthiness and `bool()`
- Short-circuiting: `and` / `or`
- Return values of `and` / `or` (not just True/False!)
- De Morgan's laws
- Guard patterns and defaults
- `any` / `all` (including edge cases)
- Bitwise vs boolean operators (&/| vs and/or)
- Conditional expression `x if cond else y`
- Custom truthiness with `__bool__` / `__len__`


## 1) Truthiness: what turns into `True` / `False`?
In Python, many non-bool objects have a truth value. Use `bool(x)` to see it.

In [1]:
samples = [
    0, 1, -1, 0.0, 0.1, "", " ", "hi", [], [0], (), (0,), {}, {1:2}, set(), {0}, None
]
table = [(repr(x), bool(x)) for x in samples]
for r in table:
    print(f"{r[0]:>10} -> {r[1]}")

assert bool(0) is False
assert bool(()) is False and bool((0,)) is True
assert bool("") is False and bool("0") is True
assert bool(None) is False
print("OK ✅")


         0 -> False
         1 -> True
        -1 -> True
       0.0 -> False
       0.1 -> True
        '' -> False
       ' ' -> True
      'hi' -> True
        [] -> False
       [0] -> True
        () -> False
      (0,) -> True
        {} -> False
    {1: 2} -> True
     set() -> False
       {0} -> True
      None -> False
OK ✅


## 2) Short-circuiting and side effects
`and` stops at the first falsy value; `or` stops at the first truthy value.

In [2]:
calls = []
def mark(name, value):
    calls.append(name)
    print("called", name)
    return value

calls.clear()
_ = mark('A', True) and mark('B', False) and mark('C', True)
assert calls == ['A','B']  # stopped before C

calls.clear()
_ = mark('X', True) or mark('Y', False)
assert calls == ['X']      # stopped before Y
print("OK ✅")


called A
called B
called X
OK ✅


## 3) Return values of `and` / `or`
They return one of the **operands**, not necessarily a `bool`.

In [3]:
# AND returns first falsy or the last value
assert (0 and 123) == 0
assert ("hi" and 42) == 42

# OR returns first truthy or the last value
assert ("" or "fallback") == "fallback"
assert (None or 0 or [] or "go") == "go"

# Useful: pick defaults
name = ""  # imagine user input
display = name or "<anonymous>"
assert display == "<anonymous>"
print("OK ✅")


OK ✅


## 4) De Morgan's laws
For any booleans `A`, `B`:
- `not (A and B)`  == `(not A) or (not B)`
- `not (A or  B)`  == `(not A) and (not B)`

In [4]:
vals = [False, True]
for A in vals:
    for B in vals:
        assert (not (A and B)) == ((not A) or (not B))
        assert (not (A or  B)) == ((not A) and (not B))
print("De Morgan OK ✅")


De Morgan OK ✅


## 5) Guard patterns with short-circuiting
Use `and` to guard a risky operation (e.g., division by zero).

In [5]:
a, b = 20, 0
safe = (b != 0) and (a / b > 1)  # right side never evaluated when b==0
assert safe is False

a, b = 20, 10
safe = (b != 0) and (a / b > 1)
assert safe is True
print("Guards OK ✅")


Guards OK ✅


## 6) `any` / `all` (and the empty-iterable edge case)
- `any(iterable)` → True if **any** element is truthy; False for empty iterables.
- `all(iterable)` → True if **all** elements are truthy; **True for empty** iterables (vacuous truth).

In [6]:
assert any([0, 0, 3]) is True
assert all([1, 2, 3]) is True
assert any([]) is False
assert all([]) is True

# With generators (short-circuits internally)
def gt10(n):
    print("test", n)
    return n > 10
res = any(gt10(n) for n in [1, 5, 12, 99])  # stops at 12
    
assert res is True
print("any/all OK ✅")


test 1
test 5
test 12
any/all OK ✅


## 7) Bitwise vs boolean operators (gotcha!)
`&` and `|` are **bitwise** (and evaluate both sides); `and`/`or` are **boolean** (and short-circuit). Precedence differs too: `&` binds tighter than `and`.

In [7]:
# With booleans, results look similar, but evaluation differs
seen = []
def T(): seen.append('T'); return True
def F(): seen.append('F'); return False

seen.clear(); _ = F() and T();  assert seen == ['F']      # short-circuit
seen.clear(); _ = F() & T();    assert seen == ['F','T']  # both evaluated

# Precedence difference
val1 = True or False and False   # True or (False and False) -> True
val2 = True | False & False      # True | (False & False) -> True
assert val1 == val2 == True
print("Bitwise vs boolean OK ✅")


Bitwise vs boolean OK ✅


## 8) Conditional expression vs `or` default
`x if cond else y` is explicit; `x or y` picks the first **truthy** value (which might not be what you want when `x` can be falsy-but-valid like `0`).

In [8]:
count_from_db = 0  # 0 is a valid value!
using_or = count_from_db or 100
using_if = count_from_db if count_from_db is not None else 100
assert using_or == 100            # picked fallback because 0 is falsy
assert using_if == 0              # kept valid 0
print("Conditional vs or OK ✅")


Conditional vs or OK ✅


## 9) Custom truthiness (`__bool__` / `__len__`)
Objects are truthy if `__bool__` returns True, or if absent, if `__len__` > 0. Otherwise falsy.

In [9]:
class Box:
    def __init__(self, items=None):
        self.items = list(items or [])
    def __len__(self):
        return len(self.items)
    def add(self, x):
        self.items.append(x)

b = Box()
assert bool(b) is False
b.add(1)
assert bool(b) is True and len(b) == 1

class Flag:
    def __init__(self, on=False):
        self.on = on
    def __bool__(self):
        return self.on

f = Flag()
assert bool(f) is False
f.on = True
assert bool(f) is True
print("Custom truthiness OK ✅")


Custom truthiness OK ✅


## 10) Quick quiz — predict before you run
Compute mentally, then verify with asserts.

In [10]:
# A) Truthiness
assert (bool([]), bool([0]), bool(""), bool("0")) == (False, True, False, True)

# B) Short-circuit and operand return values
assert ("" or None or "ok") == "ok"
assert ("go" and 123) == 123

# C) De Morgan
A, B = False, True
assert (not (A and B)) == ((not A) or (not B))
assert (not (A or B)) == ((not A) and (not B))

# D) any/all edge cases
assert any([]) is False and all([]) is True
assert any([0, 0, 1]) is True and all([1, 1, 1]) is True

# E) Bitwise vs boolean
seen = []
def T(): seen.append('T'); return True
def F(): seen.append('F'); return False
seen.clear(); _ = F() and T();  assert seen == ['F']
seen.clear(); _ = F() | T();    assert seen == ['F','T']  # bitwise OR evaluates both

# F) Conditional vs or
value = 0
assert (value or 99) == 99
assert (value if value is not None else 99) == 0

print("All quiz checks passed ✅")


All quiz checks passed ✅
